In [1]:
import pandas as pd
import requests
from datetime import datetime
from plyer import notification
import sqlite3
import ast

In [2]:
# Sistema de alerta criado
def alert(nivel, base, etapa, erro=""):
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"

    if nivel == 1:
        title = 'ATENÇÃO: Alerta de Nível Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta de Nível Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta de Nível Alto'
    else:
        print("Nivel", nivel, " não especificado ou desconhecido.")

    notification.notify(
        title=title,
        message=msg,
        app_name='Alerta',
        timeout=10
    )


In [3]:
# Função para criar tabelas
def criar_tabelas():
    conexao = sqlite3.connect('projeto_final_1-7.db')
    cursor = conexao.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pokemons (
            id INTEGER PRIMARY KEY,
            nome TEXT,
            tipo TEXT,
            altura REAL,
            peso REAL,
            experiencia_base INTEGER
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS habilidades (
            id INTEGER PRIMARY KEY,
            nome TEXT,
            efeito TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS habitats (
            id INTEGER PRIMARY KEY,
            nome TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tipos (
            id INTEGER PRIMARY KEY,
            nome TEXT,
            relacoes_dano TEXT,
            slot INTEGER
        )
    ''')

    conexao.commit()
    conexao.close()

# Chamar a função para criar as tabelas
criar_tabelas()


In [4]:
# Função para salvar no banco de dados
def save_bd(df: pd.DataFrame, nome_tabela: str):
    if df is not None:
        conect = sqlite3.connect('projeto_final_1-7.db')
        df.to_sql(nome_tabela, conect, if_exists='replace', index=False)
        conect.close()
    else:
        print(f"DataFrame para a tabela {nome_tabela} é None. Não foi possível salvar no banco de dados.")


In [5]:
# Função para carregar do banco de dados
def load_bd(nome_tabela):
    conect = sqlite3.connect('projeto_final_1-7.db')
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conect)
    conect.close()
    return df

In [6]:
# Função para obter dados da API
def get_json_api(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verifica se houve algum erro na requisição
        json_data = response.json()
        return json_data
    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer requisição para {url}: {e}")
        return None

In [7]:
# Função para extrair dados da API e salvar no banco de dados
def extract_and_save(url, table_name):
    try:
        df = pd.DataFrame()

        while url is not None:
            print(url, end='\r')

            # Obter dados da API
            json_data = get_json_api(url)

            # Para dataframe
            df_temp = pd.DataFrame(json_data['results'])

            # Append/concat
            df = pd.concat([df, df_temp], ignore_index=True)

            # Próxima página
            url = json_data.get('next')

        # Salvar no banco de dados
        save_bd(df, table_name)
        print(f"Extração de {table_name} concluída com sucesso!")

    except Exception as e:
        alert(nivel=3, base=table_name, etapa="EXTRACAO", erro=str(e))




In [8]:
# Etapa de extração
def etapa_extracao():
    try:
        extract_and_save("https://pokeapi.co/api/v2/pokemon", "pokemons_url")
        extract_and_save("https://pokeapi.co/api/v2/ability", "habilidades_url")
        extract_and_save("https://pokeapi.co/api/v2/pokemon-habitat", "habitat_url")
        extract_and_save("https://pokeapi.co/api/v2/type", "type_url")

    except Exception as e:
        alert(nivel=3, base="etapa_extracao", etapa="EXTRACAO", erro=str(e))

In [9]:
# Execute a etapa de extração
etapa_extracao()

Extração de pokemons_url concluída com sucesso!imit=20
Extração de habilidades_url concluída com sucesso!=70
Extração de habitat_url concluída com sucesso!
Extração de type_url concluída com sucesso!


In [10]:
# Função para criar a base de pokemons
def create_pokemons_base(df_pokemons_url):
    df_pokemons_full = pd.DataFrame()

    for url in df_pokemons_url['url']:
        print(url, end='\r')

        # Obter os detalhes do pokémon
        json_data = get_json_api(url)

        # Extrair informações relevantes
        id = json_data['id']
        name = json_data['name']
        abilities = [ability['ability']['name'] for ability in json_data['abilities']]
        height = json_data['height']
        weight = json_data['weight']
        base_experience = json_data['base_experience']

        # Criar uma série para o pokémon
        sr_pokemon = pd.Series({
            "id": id,
            "name": name,
            "abilities": str(abilities),  # Convertendo a lista para uma string
            "height": height,
            "weight": weight,
            "base_experience": base_experience
        })

        # Concatenar com o DataFrame geral
        df_pokemons_full = pd.concat([df_pokemons_full, sr_pokemon.to_frame().T], ignore_index=True)

    return df_pokemons_full

In [11]:
# Criação da base de pokemons
df_pokemons_url = load_bd("pokemons_url")
df_pokemons_full = create_pokemons_base(df_pokemons_url)


In [12]:
# Visualize o DataFrame
print(df_pokemons_full)

         id                      name                    abilities height  \
0         1                 bulbasaur  ['overgrow', 'chlorophyll']      7   
1         2                   ivysaur  ['overgrow', 'chlorophyll']     10   
2         3                  venusaur  ['overgrow', 'chlorophyll']     20   
3         4                charmander     ['blaze', 'solar-power']      6   
4         5                charmeleon     ['blaze', 'solar-power']     11   
...     ...                       ...                          ...    ...   
1297  10273   ogerpon-wellspring-mask             ['water-absorb']     12   
1298  10274  ogerpon-hearthflame-mask             ['mold-breaker']     12   
1299  10275  ogerpon-cornerstone-mask                   ['sturdy']     12   
1300  10276        terapagos-terastal               ['tera-shell']      3   
1301  10277         terapagos-stellar            ['teraform-zero']     17   

     weight base_experience  
0        69              64  
1       130    

In [13]:
# Função para criar a base de habilidades
def create_habilidades_base(df_habilidades_url):
    df_habilidades_full = pd.DataFrame()

    for url in df_habilidades_url['url'].values:
        json_data = get_json_api(url)

        id = json_data['id']
        name = json_data['name']
        generation = json_data['generation']['name']
        is_main_series = json_data['is_main_series']
        effect = "\n---\n".join([effect['effect'] for effect in json_data['effect_entries']])

        sr_habilidade = pd.Series({
            "id": id,
            "name": name,
            "generation": generation,
            "is_main_series": is_main_series,
            "effect": effect
        })

        df_habilidades_full = pd.concat([df_habilidades_full, sr_habilidade.to_frame().T], ignore_index=True)

    return df_habilidades_full


In [14]:

# Criação da base de habilidades
df_habilidades_url = load_bd("habilidades_url")
df_habilidades_full = create_habilidades_base(df_habilidades_url)


In [15]:
# Visualize o DataFrame
print(df_habilidades_full)


        id          name      generation is_main_series  \
0        1        stench  generation-iii           True   
1        2       drizzle  generation-iii           True   
2        3   speed-boost  generation-iii           True   
3        4  battle-armor  generation-iii           True   
4        5        sturdy  generation-iii           True   
..     ...           ...             ...            ...   
362  10056        run-up    generation-v          False   
363  10057     conqueror    generation-v          False   
364  10058       shackle    generation-v          False   
365  10059         decoy    generation-v          False   
366  10060        shield    generation-v          False   

                                                effect  
0    Attacken die Schaden verursachen haben mit jed...  
1    Das weather ändert sich zu rain wenn ein Pokém...  
2    Die speed des Pokémon erhöht sich jede Runde u...  
3    Attacken können keinen kritischen Treffer land...  
4    W

In [16]:
# Função para criar a base de habitats
def create_habitats_base(df_habitat_url):
    df_habitats_full = pd.DataFrame()

    for url in df_habitat_url['url'].values:
        try:
            habitat_data = get_json_api(url)

            if 'pokemon_species' in habitat_data:
                for species in habitat_data['pokemon_species']:
                    pokemon_name = species['name']
                    habitat_id =  habitat_data['id']
                    habitat_name = habitat_data['name']

                sr_habitat = pd.Series({
                    "id": habitat_id,
                    "name": species['name'],
                    "habitat_name": habitat_name
                })

                df_habitats_full = pd.concat([df_habitats_full, sr_habitat.to_frame().T], ignore_index=True)
            else:
                print(f"Resposta da API para URL {url} não possui as chaves 'id' e 'name'.")
        except Exception as e:
            print(f"Erro ao processar a URL {url}: {str(e)}")

    return df_habitats_full

In [17]:
# Criação da base de habitats
df_habitats_url = load_bd("habitat_url")
df_habitats_full = create_habitats_base(df_habitats_url)

In [18]:
# Visualize o DataFrame
print(df_habitats_full)

  id       name   habitat_name
0  1     glalie           cave
1  2   dusclops         forest
2  3   chimecho      grassland
3  4   medicham       mountain
4  5     deoxys           rare
5  6  metagross  rough-terrain
6  7   gorebyss            sea
7  8    banette          urban
8  9    milotic    waters-edge


In [36]:
# Função para criar a base de tipos de pokemon
def create_types_base(df_types_url):
    df_types_full = pd.DataFrame()

    for url in df_types_url['url'].values:
        try:
            type_data = get_json_api(url)

            if 'id' in type_data and 'name' in type_data:
                id = type_data['id']
                name = type_data['name']
                damage_relations = type_data.get('damage_relations', {})
                double_damage_from = damage_relations.get('double_damage_from', [])

                # Extrair a lista de 'name' de 'double_damage_from'
                names_list = [item.get('name', '') for item in double_damage_from]

                sr_types = pd.Series({
                    "id": id,
                    "name": name,
                    "double_damage_from_names": str(names_list),  # Nova coluna
                })

                df_types_full = pd.concat([df_types_full, sr_types.to_frame().T], ignore_index=True)
            else:
                print(f"Resposta da API para URL {url} não possui as chaves 'id' e 'name'.")
        except Exception as e:
            print(f"Erro ao processar a URL {url}: {str(e)}")

    return df_types_full


In [37]:
# Criação da base de tipos
df_types_url = load_bd("type_url")
df_types_full = create_types_base(df_types_url)


In [38]:
# Visualize o DataFrame
print(df_types_full)

       id      name                           double_damage_from_names
0       1    normal                                       ['fighting']
1       2  fighting                     ['flying', 'psychic', 'fairy']
2       3    flying                        ['rock', 'electric', 'ice']
3       4    poison                              ['ground', 'psychic']
4       5    ground                          ['water', 'grass', 'ice']
5       6      rock  ['fighting', 'ground', 'steel', 'water', 'grass']
6       7       bug                         ['flying', 'rock', 'fire']
7       8     ghost                                  ['ghost', 'dark']
8       9     steel                     ['fighting', 'ground', 'fire']
9      10      fire                        ['ground', 'rock', 'water']
10     11     water                              ['grass', 'electric']
11     12     grass         ['flying', 'poison', 'bug', 'fire', 'ice']
12     13  electric                                         ['ground']
13    

In [22]:
# Etapa de transformação

# Função para transformar e salvar a base de pokemons
def transform_and_save_pokemons(df_pokemons_full):
    # Converter coluna 'abilities' de lista para string
    df_pokemons_full['abilities'] = df_pokemons_full['abilities'].apply(lambda abilities: ', '.join(ast.literal_eval(abilities)))

    # Calcular a média da altura e peso
    df_pokemons_full['avg_height'] = (df_pokemons_full['height'] / 10).mean()  # Convertendo de decímetros para metros
    df_pokemons_full['avg_weight'] = (df_pokemons_full['weight'] / 10).mean()  # Convertendo de hectogramas para quilogramas

    # Criar coluna 'bmi' (Índice de Massa Corporal)
    df_pokemons_full['bmi'] = df_pokemons_full['avg_weight'] / (df_pokemons_full['avg_height'] ** 2)

    # Salvando o DataFrame transformado no banco de dados
    save_bd(df_pokemons_full, "pokemons_transformed")



In [23]:
# Executar transformação da base de pokemons
transform_and_save_pokemons(df_pokemons_full)


In [24]:
def transform_and_save_habilidades(df_habilidades_full):
    
    # Exemplo de transformação: Filtrar habilidades apenas da geração principal
    df_habilidades_full = df_habilidades_full[df_habilidades_full['is_main_series']]

    # Exemplo de transformação: Converter a coluna 'effect' para minúsculas
    df_habilidades_full['effect'] = df_habilidades_full['effect'].str.lower()

    # Após as transformações, salve o DataFrame transformado no banco de dados
    save_bd(df_habilidades_full, "habilidades_transformed")

In [25]:
# Executar transformação da base de habilidades
transform_and_save_habilidades(df_habilidades_full)

C:\Users\vitor\AppData\Local\Temp\ipykernel_22716\3891013587.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_habilidades_full['effect'] = df_habilidades_full['effect'].str.lower()


In [26]:
# Função para transformar e salvar a base de habitats
def transform_and_save_habitats(df_habitats_full):
    # Exemplo de transformação: converter o nome do habitat para maiúsculas
    df_habitats_full['habitat_name'] = df_habitats_full['habitat_name'].str.upper()

    # Salvar o DataFrame transformado no banco de dados
    save_bd(df_habitats_full, "habitats_transformed")

In [27]:
# Executar transformação da base de habitats
transform_and_save_habitats(df_habitats_full)

In [43]:
def transform_and_save_type(df_types_full):
    # Exemplo de transformação: converter o nome de quem dá dano duplo para maiúsculas
    df_types_full['double_damage_from_names'] = df_types_full['double_damage_from_names'].str.upper()

    # Salvar o DataFrame transformado no banco de dados
    save_bd(df_types_full, "type_transformed")

In [44]:
# Executar transformação da base de tipos
transform_and_save_type(df_types_full)


In [45]:
# Carregar os DataFrames transformados do banco de dados
df_pokemons_transformed = load_bd("pokemons_transformed")
df_habitats_transformed = load_bd("habitats_transformed")
df_habilidades_transformed = load_bd("habilidades_transformed")
df_type_transformed = load_bd("type_transformed")


In [52]:
# Merges em etapas para habitat, habilidades e pokemons
df_merged = pd.merge(df_pokemons_transformed[['name', 'abilities', 'avg_height', 'avg_weight', 'bmi', 'base_experience']],
                     df_habitats_transformed[['name', 'habitat_name']],
                     on='name', how='left')

df_merged = pd.merge(df_merged,
                     df_habilidades_transformed[['name', 'generation', 'is_main_series', 'effect']],
                     on='name', how='left')



In [51]:
# Visualizar o DataFrame resultante
print(df_merged)


                          name              abilities  avg_height  avg_weight  \
0                    bulbasaur  overgrow, chlorophyll    2.046774   98.086329   
1                      ivysaur  overgrow, chlorophyll    2.046774   98.086329   
2                     venusaur  overgrow, chlorophyll    2.046774   98.086329   
3                   charmander     blaze, solar-power    2.046774   98.086329   
4                   charmeleon     blaze, solar-power    2.046774   98.086329   
...                        ...                    ...         ...         ...   
1297   ogerpon-wellspring-mask           water-absorb    2.046774   98.086329   
1298  ogerpon-hearthflame-mask           mold-breaker    2.046774   98.086329   
1299  ogerpon-cornerstone-mask                 sturdy    2.046774   98.086329   
1300        terapagos-terastal             tera-shell    2.046774   98.086329   
1301         terapagos-stellar          teraform-zero    2.046774   98.086329   

            bmi  base_exper

                  pokemon_name              abilities  avg_height  avg_weight  \
0                    bulbasaur  overgrow, chlorophyll    2.046774   98.086329   
1                      ivysaur  overgrow, chlorophyll    2.046774   98.086329   
2                     venusaur  overgrow, chlorophyll    2.046774   98.086329   
3                   charmander     blaze, solar-power    2.046774   98.086329   
4                   charmeleon     blaze, solar-power    2.046774   98.086329   
...                        ...                    ...         ...         ...   
1297   ogerpon-wellspring-mask           water-absorb    2.046774   98.086329   
1298  ogerpon-hearthflame-mask           mold-breaker    2.046774   98.086329   
1299  ogerpon-cornerstone-mask                 sturdy    2.046774   98.086329   
1300        terapagos-terastal             tera-shell    2.046774   98.086329   
1301         terapagos-stellar          teraform-zero    2.046774   98.086329   

            bmi  base_exper